# CareMap: EHR Companion

## One Model. Three Modules. Maximum Impact on Both Sides of Healthcare.

---

**Kaggle Solve for India with Gemini Competition**

CareMap demonstrates how a single clinical AI model (MedGemma) can enhance healthcare on **both sides**:

| Side | Who | Module | MedGemma Mode |
|------|-----|--------|---------------|
| **Patient Side** | Caregivers, Ayahs, Families | Fridge Sheet Generator | Text Reasoning |
| **Provider Side** | Radiologists | X-ray Triage Queue | **Multimodal** |
| **Provider Side** | Lab/Clinical Staff | HL7 Message Triage | Text Reasoning |

---

## The Problem: Healthcare Information Overload

### On the Patient Side

Family caregivers receive overwhelming amounts of medical information:
- Dense clinical notes with medical jargon
- Multiple medication instructions with complex interactions
- Lab results with unexplained abbreviations
- Follow-up tasks scattered across documents

**In India specifically**, the challenge is compounded by:
- **Rotating Ayahs** (home health workers) who change frequently
- **Language barriers** - Ayahs often cannot read English
- **No EHR infrastructure** - patients carry paper records in bags
- **Overwhelmed doctors** - seeing 125-175 patients per day

### On the Provider Side

Healthcare workers are drowning in data:
- **Radiologists**: 1:100,000 patient ratio in India, 72-hour report delays
- **Lab Staff**: 1000+ HL7 messages per day, critical values buried in routine results
- **No prioritization** - life-threatening findings wait in queue behind normal studies

---

## User Research: Voices from the Field

We interviewed **4 healthcare professionals and caregivers** to understand the real challenges:

### Dr. Vinodhini Sriram (Primary Care Physician, US)

> "For the caregivers, I think it's mainly the medications. They are not going to care about the kidney function... **All they need is that first page which is the medication 8:00 a.m., 12 p.m., whatever it is.**"

> "Absolutely. If the source of this data is reliable, **this is very good.**"

### Sunayana Mann (Daughter/Family Caregiver)

> "If I had to drive her to the hospital, she cannot tell me all the medications and I might not be able to grab the whole bag. **So easiest would be... I peel the fridge magnet.**"

> "**Task oriented for them** like I don't want them to think. I want to give them instructions and I want them to follow it like nothing extra."

### Dr. Manini Moudgal (Pediatrician, Bangalore, India)

> "Given the volumes that we deal with over here, **EHR is a bloody headache**... We usually see **125-175 patients a day**."

### Dr. Gaurav Mishra (Psychiatrist, Geriatric Focus)

> "**The pages for the ayah need to be even simpler**... for kids with autism sometimes we would use image based chart."

> "I would have **both levels available**... deep dive for family, simple for ayah."

### Common Themes

1. **Two audiences** - Ayah (simple, task-oriented) vs Family Caregiver (understanding context)
2. **Medication schedule is the most critical** - everything else is secondary
3. **Simplicity is essential** - symbols, pictures, minimal text for Ayahs
4. **Don't omit medications** - print everything
5. **Emergency contacts** - who to call when something goes wrong

---

## The Hypothesis

**A single clinical AI model can serve BOTH sides of healthcare:**

```
                    ┌─────────────────────────────────┐
                    │         MedGemma 4B-IT          │
                    │   Clinical Foundation Model     │
                    │   (Text + Multimodal)           │
                    └─────────────────────────────────┘
                                   │
              ┌────────────────────┼────────────────────┐
              │                    │                    │
              ▼                    ▼                    ▼
    ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐
    │  Module 1       │  │  Module 2       │  │  Module 3       │
    │  Fridge Sheets  │  │  X-ray Triage   │  │  HL7 Triage     │
    │  (Patient Side) │  │  (Multimodal)   │  │  (Provider Side)│
    └─────────────────┘  └─────────────────┘  └─────────────────┘
           │                    │                    │
           ▼                    ▼                    ▼
    Ayahs & Families     Radiologists        Lab Staff
```

**Why MedGemma?**

MedGemma is specifically trained on clinical text including **Electronic Health Records (EHR)**, making it ideal for:
- Understanding medication sig codes (e.g., "PO BID AC" → "by mouth, twice daily, before meals")
- Interpreting lab result patterns and clinical significance
- Analyzing chest X-rays for findings and urgency
- Translating medical jargon while preserving safety-critical information

---

## Environment Setup

In [ ]:
# Install dependencies (uncomment if running on Kaggle or fresh environment)
# !pip install -q transformers>=4.39.0 accelerate>=0.27.0 huggingface-hub>=0.34.0 pydantic>=2.6.0 sentencepiece tqdm

In [1]:
import json
import os
import sys
from pathlib import Path
from typing import Dict, Any, List
from IPython.display import display, Markdown, HTML, IFrame
from tqdm import tqdm

# Add src to path for imports
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(project_root / 'src'))

print(f"Project root: {project_root}")
print(f"Python version: {sys.version}")

Project root: /Users/rumpagiri/Projects/caremap-medgemma
Python version: 3.14.2 (main, Dec  5 2025, 16:49:16) [Clang 17.0.0 (clang-1700.4.4.1)]


In [2]:
# Load MedGemma
from caremap.llm_client import MedGemmaClient

print("Loading MedGemma 4B-IT...")
print("(This may take 1-2 minutes on first run)")

client = MedGemmaClient()

print(f"\nModel: {client.model_id}")
print(f"Device: {client.device}")
print("Ready!")

Loading MedGemma 4B-IT...
(This may take 1-2 minutes on first run)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Model: google/medgemma-4b-it
Device: mps
Ready!


---

## Module 1: Patient Portal - Fridge Sheet Generator

### The Use Case: Dadu's Care Team

**Dadu** is an 82-year-old grandfather with:
- Type 2 Diabetes with kidney complications (CKD Stage 3b)
- Congestive Heart Failure
- Atrial Fibrillation (on blood thinners)

He has **8 medications** with complex interactions, **6 lab results** to monitor, and **5 care actions** to track.

His care team includes:
- **Rotating Ayahs** who may change weekly
- **His daughter** who lives in another city
- **His spouse** who is also elderly

CareMap generates **5 printable pages** tailored to each audience:

| Page | Audience | Purpose |
|------|----------|--------|
| 1. Medications | Ayah/Helper | Daily schedule with time/food badges |
| 2. Labs | Family Caregiver | Test results in plain language |
| 3. Care Actions | Family Caregiver | Today/Week/Later task buckets |
| 4. Imaging | Family Caregiver | X-ray findings explained |
| 5. Connections | Both | How meds, labs, and actions connect |

In [3]:
# Load Dadu's patient data
golden_file = project_root / 'examples' / 'golden_patient_complex.json'

with open(golden_file) as f:
    patient_data = json.load(f)

print(f"Patient: {patient_data['patient']['nickname']}")
print(f"Age: {patient_data['patient']['age_range']}")
print(f"Conditions: {', '.join(patient_data['patient']['conditions_display'])}")
print(f"\nMedications: {len(patient_data['medications'])}")
print(f"Lab Results: {len(patient_data['results'])}")
print(f"Care Gaps: {len(patient_data['care_gaps'])}")

Patient: Dadu
Age: 80s
Conditions: Type 2 Diabetes with kidney complications, Congestive Heart Failure, Atrial Fibrillation

Medications: 8
Lab Results: 6
Care Gaps: 5


### 1.1 Medication Interpretation

MedGemma transforms complex medication instructions into plain language:

**Input (from EHR):**
```
Warfarin - Take as directed based on INR results
Clinician Notes: Target INR 2.0-3.0 for AFib. Weekly INR checks required.
Interaction Notes: Avoid NSAIDs. Keep vitamin K intake consistent.
```

**Output (for Ayah):**
- **What it does:** Helps prevent dangerous blood clots
- **When:** Evening, same time each day
- **Watch for:** Unusual bleeding, bruising

In [4]:
from caremap.medication_interpretation import interpret_medication_v3_grounded

print("Interpreting medications with MedGemma...")
print("=" * 60)

# Interpret first 3 medications as examples
for med in tqdm(patient_data['medications'][:3], desc="Medications"):
    result, _ = interpret_medication_v3_grounded(
        client=client,
        medication_name=med['medication_name'],
        sig_text=med['sig_text'],
        clinician_notes=med.get('clinician_notes', ''),
        interaction_notes=med.get('interaction_notes', ''),
    )
    
    print(f"\n{'='*60}")
    print(f"MEDICATION: {med['medication_name']} ({med['timing']})")
    print(f"{'='*60}")
    if 'raw_response' not in result:
        print(f"What this does: {result.get('what_this_does', 'N/A')}")
        print(f"Watch out for: {result.get('watch_out_for', 'N/A')}")

Interpreting medications with MedGemma...


Medications:  33%|████████████████████████████                                                        | 1/3 [00:09<00:19,  9.88s/it]


MEDICATION: Metformin (morning and evening, with food)
What this does: This medicine helps lower your blood sugar. It helps your body use sugar better.
Watch out for: If a test shows your kidneys aren't working well, stop taking this medicine. Also, stop taking it 48 hours before and after any CT scan with contrast dye. Talk to your doctor if you have any questions.


Medications:  67%|████████████████████████████████████████████████████████                            | 2/3 [00:18<00:09,  9.03s/it]


MEDICATION: Warfarin (evening, same time each day)
What this does: This medication helps prevent dangerous blood clots. It is often called a 'blood thinner' because it makes the blood take longer to clot.
Watch out for: Do not take NSAIDs like ibuprofen or aspirin. Try to keep the amount of leafy green vegetables (like spinach or kale) in the diet the same each week. Tell your doctor if you have any unusual bleeding or bruising.


Medications: 100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:25<00:00,  8.64s/it]


MEDICATION: Furosemide (morning and early afternoon, not after 4pm)
What this does: This medicine helps your body get rid of extra fluid. It's often used to treat heart failure and swelling.
Watch out for: Weigh yourself every day. If you feel dizzy or lightheaded, tell your doctor. This medicine can sometimes lower your potassium levels, so take the potassium supplement your doctor prescribed.


### 1.2 Lab Interpretation

MedGemma explains lab results without numeric values or medical jargon:

**Design Principle:** Caregivers don't need to know "INR = 3.5". They need to know:
- What the test checks
- Whether the result is concerning
- What question to ask the doctor

In [5]:
from caremap.lab_interpretation import interpret_lab

print("Interpreting lab results with MedGemma...")
print("=" * 60)

for lab in tqdm(patient_data['results'][:3], desc="Labs"):
    result = interpret_lab(
        client=client,
        test_name=lab['test_name'],
        meaning_category=lab['meaning_category'],
        source_note=lab.get('source_note', ''),
    )
    
    print(f"\n{'='*60}")
    print(f"TEST: {lab['test_name']} ({lab['flag']})")
    print(f"{'='*60}")
    print(f"What was checked: {result.get('what_was_checked', 'N/A')}")
    print(f"What it means: {result.get('what_it_means', 'N/A')}")
    print(f"Ask the doctor: {result.get('what_to_ask_doctor', 'N/A')}")

Interpreting lab results with MedGemma...


Labs:  33%|██████████████████████████████▎                                                            | 1/3 [00:07<00:14,  7.41s/it]


TEST: INR (Blood Thinner Level) (high)
What was checked: A test that checks how well your blood clots.
What it means: Needs follow-up. This test result is above the target range, which means your blood is clotting too much. Your doctor may need to adjust your blood thinner medication.
Ask the doctor: Could you help us understand why the INR is above the target range and what adjustments need to be made to the warfarin?


Labs:  67%|████████████████████████████████████████████████████████████▋                              | 2/3 [00:13<00:06,  6.68s/it]


TEST: Kidney Function (eGFR) (low)
What was checked: A test that measures how well your kidneys are working.
What it means: Slightly off. This means your kidney function is not quite where your doctor wants it to be, but it's not a major concern. It's important to continue monitoring.
Ask the doctor: Could you help us understand what factors might be affecting my kidney function?


Labs: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:18<00:00,  6.22s/it]


TEST: Potassium Level (normal)
What was checked: A test that measures the amount of potassium in your blood.
What it means: Normal. This means your potassium level is within the healthy range.
Ask the doctor: Could you help us understand what a normal potassium level is for my specific situation?


### 1.3 Care Gap Interpretation

MedGemma converts clinical care gaps into actionable tasks:

**Input:** "INR recheck needed - last result was high"

**Output:**
- **Action:** Call clinic to schedule INR blood draw
- **Why:** Blood thinner level needs monitoring
- **Urgency:** This Week

In [6]:
from caremap.caregap_interpretation import interpret_caregap

print("Interpreting care gaps with MedGemma...")
print("=" * 60)

for gap in tqdm(patient_data['care_gaps'][:3], desc="Care Gaps"):
    result = interpret_caregap(
        client=client,
        item_text=gap['item_text'],
        next_step=gap['next_step'],
        time_bucket=gap['time_bucket'],
    )
    
    print(f"\n{'='*60}")
    print(f"GAP: {gap['item_text']} [{gap['time_bucket']}]")
    print(f"{'='*60}")
    print(f"Time bucket: {result.get('time_bucket', 'N/A')}")
    print(f"Action: {result.get('action_item', 'N/A')}")
    print(f"Next step: {result.get('next_step', 'N/A')}")

Interpreting care gaps with MedGemma...


Care Gaps:  33%|████████████████████████████▋                                                         | 1/3 [00:03<00:06,  3.46s/it]


GAP: INR recheck needed - last result was high [This Week]
Time bucket: This Week
Action: Call clinic to schedule INR blood draw
Next step: Call clinic to schedule INR blood draw


Care Gaps:  67%|█████████████████████████████████████████████████████████▎                            | 2/3 [00:06<00:03,  3.18s/it]


GAP: Diabetic eye exam overdue by 8 months [This Week]
Time bucket: This Week
Action: Schedule an eye exam
Next step: Schedule appointment with eye doctor


Care Gaps: 100%|██████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:09<00:00,  3.18s/it]


GAP: Annual flu shot not received [Today]
Time bucket: Today
Action: Ask pharmacy about flu shot availability
Next step: Ask pharmacy about flu shot availability


### 1.4 Imaging Interpretation

For heart failure patients like Dadu, chest X-rays reveal important information about fluid buildup. MedGemma can explain findings in plain language while connecting them to the patient's medications.

**Safety Constraint:** CareMap never uses the word "cancer" - that's for the oncologist.

In [7]:
from caremap.imaging_interpretation import interpret_imaging_report

# Load a CT scenario for a heart failure patient
ct_file = project_root / 'examples' / 'golden_imaging_ct.json'
with open(ct_file) as f:
    ct_data = json.load(f)

ct_scenario = ct_data['ct_scenarios'][0]  # CHF patient with shortness of breath

print(f"Study: {ct_scenario['study_type']}")
print(f"Indication: {ct_scenario['clinical_indication']}")
print(f"\nRadiology Report (raw):")
print(f"{ct_scenario['radiology_report']['impression']}")

print("\nInterpreting with MedGemma...")

imaging_result = interpret_imaging_report(
    client=client,
    study_type=ct_scenario['study_type'],
    report_text=ct_scenario['radiology_report']['impression'],
    flag=ct_scenario['flag'],
)

print(f"\n{'='*60}")
print(f"IMAGING INTERPRETATION (Plain Language)")
print(f"{'='*60}")
print(f"What was done: {imaging_result.get('what_was_done', 'N/A')}")
print(f"Key finding: {imaging_result.get('key_finding', 'N/A')}")
print(f"Ask the doctor: {imaging_result.get('what_to_ask_doctor', 'N/A')}")

Study: CT Chest with Contrast
Indication: Shortness of breath, rule out pulmonary embolism

Radiology Report (raw):
1. No pulmonary embolism. 2. Small bilateral pleural effusions with bibasilar atelectasis. 3. Cardiomegaly. Clinical correlation recommended.

Interpreting with MedGemma...

IMAGING INTERPRETATION (Plain Language)
What was done: The scan looked at the lungs, heart, and fluid around the lungs.
Key finding: There are some small areas of fluid around the lungs and some areas of the lungs that are collapsed. The heart is also a bit larger than normal. The doctors will review these findings to see if they need further attention.
Ask the doctor: When will we know more about these findings?


### 1.5 Connections: The Big Picture

CareMap helps caregivers understand **why** things are connected:

```
Warfarin (blood thinner) ──→ INR Test ──→ Weekly Monitoring
         │                      │
         └── Avoid NSAIDs ──────┘
```

This is especially valuable when explaining care to new Ayahs.

### 1.6 Generate Complete Fridge Sheets (5 Pages)

Now we generate all 5 printable HTML pages:

In [8]:
from caremap.fridge_sheet_html import (
    generate_medications_page,
    generate_labs_page,
    generate_gaps_page,
    generate_imaging_page,
    generate_connections_page,
    PatientInfo,
)

# Create patient info
patient_info = PatientInfo(
    nickname=patient_data['patient']['nickname'],
    age_range=patient_data['patient']['age_range'],
    conditions=patient_data['patient']['conditions_display']
)

# Output directory
html_output_dir = project_root / 'outputs' / 'fridge_sheets'
html_output_dir.mkdir(parents=True, exist_ok=True)

print(f"Generating 5 Concept B pages for {patient_info.nickname}...")
print(f"Output: {html_output_dir}")

Generating 5 Concept B pages for Dadu...
Output: /Users/rumpagiri/Projects/caremap-medgemma/outputs/fridge_sheets


In [9]:
# Page 1: Medications (for Ayah)
print("\n[1/5] Generating MEDICATIONS page (for Ayah)...")

meds_html = generate_medications_page(
    patient=patient_info,
    medications=patient_data['medications'],
    client=client,
    page_num=1,
    total_pages=5,
    progress_callback=lambda c, t, m: print(f"      [{c}/{t}] {m}")
)

med_file = html_output_dir / '1_medications.html'
with open(med_file, 'w') as f:
    f.write(meds_html)
print(f"      Saved: {med_file.name}")


[1/5] Generating MEDICATIONS page (for Ayah)...
      [1/8] Metformin
      [2/8] Warfarin
      [3/8] Furosemide
      [4/8] Potassium Chloride
      [5/8] Carvedilol
      [6/8] Lisinopril
      [7/8] Levothyroxine
      [8/8] Acetaminophen
      Saved: 1_medications.html


In [10]:
# Page 2: Labs (for Family)
print("\n[2/5] Generating LABS page (for Family)...")

labs_html = generate_labs_page(
    patient=patient_info,
    results=patient_data['results'],
    client=client,
    page_num=2,
    total_pages=5,
    progress_callback=lambda c, t, m: print(f"      [{c}/{t}] {m}")
)

labs_file = html_output_dir / '2_labs.html'
with open(labs_file, 'w') as f:
    f.write(labs_html)
print(f"      Saved: {labs_file.name}")


[2/5] Generating LABS page (for Family)...
      [1/6] INR (Blood Thinner Level)
      [2/6] Kidney Function (eGFR)
      [3/6] Potassium Level
      [4/6] Hemoglobin A1c (3-Month Blood Sugar Average)
      [5/6] BNP (Heart Strain Marker)
      [6/6] TSH (Thyroid Function)
      Saved: 2_labs.html


In [11]:
# Page 3: Care Gaps (for Family)
print("\n[3/5] Generating CARE GAPS page (for Family)...")

gaps_html = generate_gaps_page(
    patient=patient_info,
    care_gaps=patient_data['care_gaps'],
    client=client,
    page_num=3,
    total_pages=5,
    progress_callback=lambda c, t, m: print(f"      [{c}/{t}] {m}")
)

gaps_file = html_output_dir / '3_care_gaps.html'
with open(gaps_file, 'w') as f:
    f.write(gaps_html)
print(f"      Saved: {gaps_file.name}")


[3/5] Generating CARE GAPS page (for Family)...
      [1/5] INR recheck needed - last result was high
      [2/5] Diabetic eye exam overdue by 8 months
      [3/5] Annual flu shot not received
      [4/5] Daily weight check for fluid monitoring
      [5/5] Cardiology follow-up appointment due
      Saved: 3_care_gaps.html


In [ ]:
# Page 4: Imaging (with X-ray from NIH dataset)
print("\n[4/5] Generating IMAGING page with chest X-ray...")

xray_path = project_root / 'data' / 'nih_chest_xray' / 'demo_images' / 'stat' / '00000032_001.png'
print(f"      Using X-ray: {xray_path.name}")

imaging_html = generate_imaging_page(
    patient=patient_info,
    image_path=str(xray_path) if xray_path.exists() else None,
    client=client,
    page_num=4,
    total_pages=5,
    progress_callback=lambda c, t, m: print(f"      [{c}/{t}] {m}")
)

imaging_file = html_output_dir / '4_imaging.html'
with open(imaging_file, 'w') as f:
    f.write(imaging_html)
print(f"      Saved: {imaging_file.name}")

In [ ]:
# Page 5: Connections (for Both)
print("\n[5/5] Generating CONNECTIONS page (for Both)...")

connections_html = generate_connections_page(
    patient=patient_info,
    medications=patient_data['medications'],
    results=patient_data['results'],
    care_gaps=patient_data['care_gaps'],
    contacts=patient_data['contacts'],
    page_num=5,
    total_pages=5,
    progress_callback=lambda c, t, m: print(f"      [{c}/{t}] {m}")
)

connections_file = html_output_dir / '5_connections.html'
with open(connections_file, 'w') as f:
    f.write(connections_html)
print(f"      Saved: {connections_file.name}")

print("\n" + "=" * 60)
print("All 5 fridge sheet pages generated!")
print("=" * 60)

In [ ]:
# Preview the Medications page
print("Preview: Page 1 - Medications (for Ayah)")
display(IFrame(src=str(med_file), width='100%', height=600))

### 1.7 Multilingual Support: Bengali Fridge Sheet

For Ayahs who cannot read English, CareMap can translate the fridge sheet using Meta's NLLB-200 model:

In [ ]:
from caremap.translation import NLLBTranslator

print("Loading NLLB-200 translator...")
translator = NLLBTranslator()
print("Translator ready!")

# Example: Translate medication instructions to Bengali
sample_text = "This medicine helps control your heart rate. Take it every day at the same time. Do NOT stop taking it suddenly."

print(f"\nOriginal (English):")
print(f"  {sample_text}")

bengali = translator.translate_to(sample_text, "ben_Beng")
print(f"\nBengali (for Ayah):")
print(f"  {bengali}")

---

## Module 2: Radiology Triage (Multimodal)

### The Problem

| Metric | India | Impact |
|--------|-------|--------|
| Radiologist-to-patient ratio | 1:100,000 | Severe shortage |
| Average X-ray report delay | 72 hours | Critical findings missed |
| Daily imaging volume | 500+ studies/radiologist | Burnout, errors |

### The Solution: MedGemma Multimodal Triage

MedGemma analyzes chest X-rays and assigns priority:

| Priority | Review Time | Examples |
|----------|-------------|----------|
| STAT | < 1 hour | Pulmonary edema, pneumothorax |
| SOON | < 24 hours | Cardiomegaly, consolidation |
| ROUTINE | 48-72 hours | Normal, minor findings |

**Key Principle:** AI prioritizes the worklist. Radiologists review ALL images.

In [ ]:
# Load MedGemma with multimodal support
print("Loading MedGemma with multimodal support...")
multimodal_client = MedGemmaClient(enable_multimodal=True)
print(f"Device: {multimodal_client.device}")
print("Ready for image analysis!")

In [ ]:
# Load sample X-ray from NIH dataset
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

demo_images_dir = project_root / 'data' / 'nih_chest_xray' / 'demo_images'
manifest_path = project_root / 'data' / 'nih_chest_xray' / 'sample_manifest.csv'
manifest = pd.read_csv(manifest_path)

# Display a STAT case
stat_image = manifest[manifest['priority'] == 'STAT'].iloc[0]
image_path = demo_images_dir / 'stat' / stat_image['image_id']

print(f"STAT Case: {stat_image['image_id']}")
print(f"Patient: {stat_image['patient_age']} year old {stat_image['patient_gender']}")
print(f"Ground Truth: {stat_image['findings']}")

img = Image.open(image_path)
plt.figure(figsize=(8, 8))
plt.imshow(img, cmap='gray')
plt.title(f"Chest X-ray: {stat_image['image_id']}")
plt.axis('off')
plt.show()

In [ ]:
from caremap.radiology_triage import analyze_xray

print("Analyzing X-ray with MedGemma multimodal...")

result = analyze_xray(
    client=multimodal_client,
    image_path=str(image_path),
    patient_age=int(stat_image['patient_age']),
    patient_gender=stat_image['patient_gender']
)

priority_emoji = {'STAT': '\U0001F534', 'SOON': '\U0001F7E1', 'ROUTINE': '\U0001F7E2'}
priority = result.get('priority', 'STAT')

print(f"\n{priority_emoji.get(priority, '')} MedGemma Triage Result: {priority}")
print(f"Confidence: {result.get('confidence', 0):.0%}")
print(f"\nFindings:")
for finding in result.get('findings', []):
    print(f"  - {finding}")
print(f"\nGround Truth: {stat_image['findings']}")

---

## Module 3: HL7 ORU Message Triage (Text)

### The Problem

Clinical staff receive **1000+ HL7 messages per day**. Critical values (K+ > 6.5, Troponin elevation) can be buried in routine results.

### The Solution: MedGemma Text Reasoning

MedGemma triages incoming lab results and clinical messages by urgency.

In [ ]:
from caremap.hl7_triage import load_sample_messages, triage_oru_message

messages = load_sample_messages()
stat_msg = [m for m in messages if m['expected_priority'] == 'STAT'][0]

print(f"Sample STAT Message: {stat_msg['message_id']}")
print(f"Patient: {stat_msg['patient']['age']}yo {stat_msg['patient']['gender']}")
print(f"Context: {stat_msg['clinical_context']}")
print(f"\nObservations:")
for obs in stat_msg['observations']:
    print(f"  - {obs['test_name']}: {obs['value']} {obs.get('units', '')} (ref: {obs['reference_range']})")

In [ ]:
print("\nTriaging with MedGemma...")

result = triage_oru_message(client, stat_msg)

priority = result.get('priority', 'STAT')
print(f"\n{priority_emoji.get(priority, '')} Triage Result: {priority}")
print(f"Reason: {result.get('priority_reason', 'N/A')}")
print(f"Recommended Action: {result.get('recommended_action', 'N/A')}")

---

## Impact Summary

### One Model, Maximum Impact

| Module | Beneficiary | Impact |
|--------|-------------|--------|
| **Fridge Sheets** | Caregivers & Ayahs | Reduced confusion, fewer medication errors |
| **Radiology Triage** | Radiologists | Critical findings prioritized, faster diagnosis |
| **HL7 Triage** | Lab/Clinical Staff | Life-threatening values surfaced immediately |

### Why This Matters for India

- **Caregivers**: Rotating Ayahs can follow clear instructions without verbal handoffs
- **Radiologists**: Scarce specialists can focus on urgent cases first
- **Patients**: Better outcomes through timely intervention

---

## Model Attribution

```bibtex
@article{sellergren2025medgemma,
  title={MedGemma Technical Report},
  author={Sellergren, Andrew and others},
  journal={arXiv preprint arXiv:2507.05201},
  year={2025}
}
```

---

**Interactive Demo:** [CareMap on HuggingFace Spaces](https://huggingface.co/spaces/codekunoichi/caremap-medgemma)

*CareMap - Making healthcare information accessible and actionable.*